In [7]:
import pandas as pd

In [8]:
data = pd.read_csv("../creditcard.csv")
data = pd.DataFrame(data)

In [9]:
# data.isna().sum() = Conta a quantidade de valores NAN

n_transacoes = data['Class'].count()
n_fraudes = data['Class'].sum()
n_normais = n_transacoes - n_fraudes
fraudes_porc = n_fraudes / n_transacoes
nomais_porc = n_normais / n_transacoes

# "%.2f" %(fraudes_porc*100) = Deixa o resultado com 2 casas decimais

print("Número de transações: ", n_transacoes)
print("Número de fraudes: ", n_fraudes, "%.2f" %(fraudes_porc*100))
print("Número de transações normais", n_normais, "%.2f" %(nomais_porc*100))

Número de transações:  284807
Número de fraudes:  492 0.17
Número de transações normais 284315 99.83


In [10]:
from sklearn.model_selection import StratifiedShuffleSplit

def executar_validador(x, y):
    validador = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)

    for treino_id , teste_id in validador.split(x, y):
        x_train, x_test = x[treino_id], x[teste_id]
        y_train, y_test = y[treino_id], y[teste_id]

    return x_train, x_test, y_train, y_test

In [11]:
%%time
from sklearn import tree

def executar_classificador(classificador, x_train, x_test, y_train):
    arvore = classificador.fit(x_train, y_train)
    y_pred = arvore.predict(x_test)
    return y_pred

CPU times: total: 0 ns
Wall time: 0 ns


In [12]:
import matplotlib.pyplot as plt

def salvar_arvore(classificador, nome):
    plt.figure(figsize=(200,100))
    tree.plot_tree(classificador, filled=True, fontsize=14)
    plt.savefig(nome)
    plt.close()

In [13]:
# Execução do validador
x = data.drop('Class', axis=1).values
y = data['Class'].values

x_train, x_test, y_train, y_test = executar_validador(x, y)

In [14]:
# execução do classificador DecisionTreeClassifier
classificador_arvore_decisao = tree.DecisionTreeClassifier()
y_pred_arvore_decisao = executar_classificador(classificador_arvore_decisao, x_train, x_test, y_train)

In [15]:
# Criação da figura da arvore de decisão
salvar_arvore(classificador_arvore_decisao, "arvore_decisao1.png")